In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry
import datetime
import time
from collections import Counter
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from intervals import Interval
import warnings
warnings.filterwarnings('ignore')

## 垃圾数据处理

In [5]:
# 数据预处理
data=pd.read_csv('./data/Hoorn_OpenLitterMap.csv')

In [6]:
df = data.fillna(0)# 填充缺失值

In [7]:
df

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,...,ear_plugs,elec_small,elec_large,batteries,balloons.1,life_buoy,other,custom_tag_1,custom_tag_2,custom_tag_3
0,118956,2,iPhone 6,2021-02-03 14:05:54,52.651031,5.081417,Yes,"1, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
1,118957,2,iPhone 6,2021-02-03 13:54:23,52.649972,5.082583,Yes,"18, Amstelweg, Kersenboogerd, Hoorn, North Hol...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
2,118958,2,iPhone 6,2021-02-03 14:06:30,52.650917,5.081553,Yes,"2, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
3,118959,2,iPhone 6,2021-02-03 14:02:00,52.651183,5.082656,Yes,"47, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
4,118960,2,iPhone 6,2021-02-03 14:01:17,52.651181,5.082553,Yes,"29, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19622,279325,2,iPhone 13 mini,2022-03-20 09:46:20,52.652412,5.081511,Yes,"43, Ekster, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
19623,279326,2,iPhone 13 mini,2022-03-20 09:45:09,52.652428,5.080361,Yes,"74, Lijster, Kersenboogerd, Hoorn, North Holla...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
19624,281636,2,iPhone 13 mini,2022-03-20 10:00:23,52.654053,5.079644,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,brand:oasis,0,0.0
19625,281638,2,iPhone 13 mini,2022-03-20 09:59:52,52.654020,5.079669,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0


### 统计

In [9]:
sum(df['total_litter'])

21789.0

### 垃圾分类

In [10]:
# 垃圾分类
def sumc(x):# 定义合并类的函数
    count_sum = 0
    for k in x.iteritems():
        count_sum += k[1]
    return count_sum

In [11]:
df['SMOKING']=df.loc[:,'butts':'smokingOther'].apply(sumc,axis=1)
df['SMOKING'].sum()

1074.0

In [12]:
df=df.drop(df.loc[:,'butts':'smokingOther'],axis=1)

In [13]:
df['FOOD']=df.loc[:,'sweetWrappers':'chewing_gum'].apply(sumc,axis=1)
df['FOOD'].sum()

2579.0

In [14]:
df=df.drop(df.loc[:,'sweetWrappers':'chewing_gum'],axis=1)

In [15]:
df['COFFEE']=df.loc[:,'coffeeCups':'coffeeOther'].apply(sumc,axis=1)
df['COFFEE'].sum()

559.0

In [16]:
df=df.drop(df.loc[:,'coffeeCups':'coffeeOther'],axis=1)

In [17]:
df['ALCOHOL']=df.loc[:,'beerCan':'alcoholOther'].apply(sumc,axis=1)
df['ALCOHOL'].sum()

1276.0

In [18]:
df=df.drop(df.loc[:,'beerCan':'alcoholOther'],axis=1)

In [19]:
df['SOFTDRINKS']=df.loc[:,'waterBottle':'softDrinkOther'].apply(sumc,axis=1)
df['SOFTDRINKS'].sum()

6208.0

In [20]:
df=df.drop(df.loc[:,'waterBottle':'softDrinkOther'],axis=1)

In [21]:
df['SANITARY']=df.loc[:,'condoms':'hand_sanitiser'].apply(sumc,axis=1)
df['SANITARY'].sum()

997.0

In [22]:
df=df.drop(df.loc[:,'condoms':'hand_sanitiser'],axis=1)

In [23]:
df['COASTAL']=df.loc[:,'microplastics':'styro_large'].apply(sumc,axis=1)
df['COASTAL'].sum()

327.0

In [24]:
df=df.drop(df.loc[:,'microplastics':'styro_large'],axis=1)

In [25]:
df['DUMPING']=df.loc[:,'small':'large'].apply(sumc,axis=1)
df['DUMPING'].sum()

0.0

In [26]:
df=df.drop(df.loc[:,'small':'large'],axis=1)

In [27]:
df['INDUSTRIAL']=df.loc[:,'oil':'industrial_other'].apply(sumc,axis=1)
df['INDUSTRIAL'].sum()

0.0

In [28]:
df['BRANDS']=df.loc[:,'aadrink':'wrigleys'].apply(sumc,axis=1)
df['BRANDS'].sum()

4751.0

In [29]:
df=df.drop(df.loc[:,'aadrink':'wrigleys'],axis=1)

In [30]:
df=df.drop(df.loc[:,'oil':'industrial_other'],axis=1)

In [31]:
df['DOGSHIT']=df.loc[:,'poo':'poo_in_bag'].apply(sumc,axis=1)
df['DOGSHIT'].sum()

0.0

In [32]:
df=df.drop(df.loc[:,'poo':'poo_in_bag'],axis=1)

In [33]:
df['ART']=df.loc[:,'item']

In [34]:
df['ART'].sum()

0.0

In [35]:
# df['MATERIAL']=df.loc[:,'aluminium':'wood'].apply(sumc,axis=1)

In [36]:
# df['MATERIAL'].sum()

In [37]:
# df=df.drop(df.loc[:,'aluminium':'wood'],axis=1)

In [38]:
df=df.drop('item',axis=1)

In [39]:
df['OTHER']=df.loc[:,'random_litter':'other'].apply(sumc,axis=1)
df['OTHER'].sum()

7309.0

In [40]:
df=df.drop(df.loc[:,'random_litter':'other'],axis=1)

In [41]:
df.columns

Index(['id', 'verification', 'phone', 'datetime', 'lat', 'lon', 'picked up',
       'address', 'total_litter', 'SMOKING', 'FOOD', 'COFFEE', 'ALCOHOL',
       'SOFTDRINKS', 'SANITARY', 'COASTAL', 'DUMPING', 'INDUSTRIAL', 'BRANDS',
       'DOGSHIT', 'ART', 'OTHER', 'custom_tag_1', 'custom_tag_2',
       'custom_tag_3'],
      dtype='object')

In [42]:
df=df.drop(df.loc[:,'custom_tag_1':'custom_tag_3'],axis=1)# 删除不需要的标签

In [43]:
df['OTHER']=df['OTHER']+df['ART']+df['DOGSHIT']+df['INDUSTRIAL']+df['DUMPING']

In [44]:
df['OTHER'].sum()

7309.0

In [45]:
df=df.drop(['ART','DOGSHIT','INDUSTRIAL','DUMPING'],axis=1)

In [46]:
df # 分类完毕

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
0,118956,2,iPhone 6,2021-02-03 14:05:54,52.651031,5.081417,Yes,"1, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,118957,2,iPhone 6,2021-02-03 13:54:23,52.649972,5.082583,Yes,"18, Amstelweg, Kersenboogerd, Hoorn, North Hol...",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118958,2,iPhone 6,2021-02-03 14:06:30,52.650917,5.081553,Yes,"2, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,118959,2,iPhone 6,2021-02-03 14:02:00,52.651183,5.082656,Yes,"47, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,118960,2,iPhone 6,2021-02-03 14:01:17,52.651181,5.082553,Yes,"29, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19622,279325,2,iPhone 13 mini,2022-03-20 09:46:20,52.652412,5.081511,Yes,"43, Ekster, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
19623,279326,2,iPhone 13 mini,2022-03-20 09:45:09,52.652428,5.080361,Yes,"74, Lijster, Kersenboogerd, Hoorn, North Holla...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19624,281636,2,iPhone 13 mini,2022-03-20 10:00:23,52.654053,5.079644,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19625,281638,2,iPhone 13 mini,2022-03-20 09:59:52,52.654020,5.079669,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [47]:
df=df[-df['datetime'].str.contains('2019')].reset_index(drop=True) # 删除2019年的零散数据

In [48]:
df=df[-df['datetime'].str.contains('2020')].reset_index(drop=True) # 删除2020年的零散数据

In [49]:
# df=df[-df['date_taken'].str.contains('2021')].reset_index(drop=True) # 删除2021年的零散数据

In [50]:
df=df[-df['datetime'].str.contains('2022')].reset_index(drop=True) # 删除2022年的零散数据

In [51]:
df=df[-df['datetime'].str.contains('2023')].reset_index(drop=True) # 删除2023年的零散数据

In [52]:
df

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
0,118956,2,iPhone 6,2021-02-03 14:05:54,52.651031,5.081417,Yes,"1, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,118957,2,iPhone 6,2021-02-03 13:54:23,52.649972,5.082583,Yes,"18, Amstelweg, Kersenboogerd, Hoorn, North Hol...",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118958,2,iPhone 6,2021-02-03 14:06:30,52.650917,5.081553,Yes,"2, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,118959,2,iPhone 6,2021-02-03 14:02:00,52.651183,5.082656,Yes,"47, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,118960,2,iPhone 6,2021-02-03 14:01:17,52.651181,5.082553,Yes,"29, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19577,278719,2,iPhone 6,2021-02-05 14:01:29,52.649053,5.086322,Yes,"71, Patio, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19578,278720,2,iPhone 6,2021-02-05 14:15:59,52.649753,5.082647,Yes,"20, Amstelweg, Kersenboogerd, Hoorn, North Hol...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19579,278721,2,iPhone 6,2021-03-05 12:17:58,52.655544,5.078928,Yes,"13, Lingeweg, Kersenboogerd, Hoorn, North Holl...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19580,278722,2,iPhone 6,2021-03-18 14:01:54,52.649314,5.085933,Yes,"68, Patio, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [48]:
# df.to_csv('./outdata/Hoorn_preliminary.csv')

### 合并时间

In [49]:
# df=df.rename(columns={'date_taken':'datetime'})

In [50]:
df['datetime']=pd.to_datetime(df['datetime'])

In [51]:
df['datetime']

0       2021-02-03 14:05:54
1       2021-02-03 13:54:23
2       2021-02-03 14:06:30
3       2021-02-03 14:02:00
4       2021-02-03 14:01:17
                ...        
19577   2021-02-05 14:01:29
19578   2021-02-05 14:15:59
19579   2021-03-05 12:17:58
19580   2021-03-18 14:01:54
19581   2021-01-09 12:09:42
Name: datetime, Length: 19582, dtype: datetime64[ns]

In [52]:
Date=df['datetime'].dt.date
df_dateconcat=df.loc[:,'total_litter':'OTHER'].groupby(Date).sum() # 合并同日期的垃圾数量
df_dateconcat

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
datetime,,,,,,,,,,
2021-01-06,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2021-01-07,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2021-01-08,114.0,2.0,13.0,6.0,3.0,34.0,9.0,0.0,25.0,32.0
2021-01-09,58.0,3.0,11.0,1.0,0.0,12.0,3.0,0.0,11.0,21.0
2021-01-10,49.0,2.0,2.0,0.0,8.0,20.0,0.0,0.0,18.0,9.0
...,...,...,...,...,...,...,...,...,...,...
2021-12-17,140.0,31.0,14.0,5.0,1.0,36.0,8.0,0.0,22.0,45.0
2021-12-18,18.0,0.0,6.0,0.0,3.0,6.0,1.0,0.0,6.0,2.0
2021-12-19,92.0,19.0,10.0,1.0,5.0,28.0,7.0,0.0,28.0,22.0


In [49]:
df_dateconcat.max()

total_litter    350.0
SMOKING          86.0
FOOD             43.0
COFFEE           15.0
ALCOHOL          34.0
SOFTDRINKS       82.0
SANITARY         25.0
COASTAL          38.0
BRANDS           68.0
OTHER           190.0
dtype: float64

### 对缺失的日期补0（舍弃）

In [53]:
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2022, 12, 31)
date_range = pd.date_range(start, end)

In [54]:
date_range

DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
               '2022-01-09', '2022-01-10',
               ...
               '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

In [55]:
# 筛选缺失的日期
df_date=pd.to_datetime(df_dateconcat.index) 
dateflag=date_range.isin(df_date) 

In [56]:
# date_all=np.concatenate((pd.Series(df_dateconcat.index).values,pd.Series(date_range[~dateflag].date).values),axis=0)# 合并两个时间
# date_all.size
# date_all与date_range相同，弃用data_all

In [57]:
Counter(dateflag)

Counter({True: 249, False: 116})

In [58]:
# 缺失的数据
dfdif=pd.DataFrame(np.zeros(116*10).reshape(116,10),
                   index=[pd.Series(date_range[~dateflag].date).values],
                   columns=['total_litter','SMOKING','FOOD','COFFEE','ALCOHOL','SOFTDRINKS','SANITARY','BRANDS','MATERIAL','OTHER'])

In [59]:
df_dateall=pd.concat([df_dateconcat,dfdif])# 合并

In [60]:
df_dateall=df_dateall.loc[:,'total_litter':'OTHER'].groupby(date_range).sum()

In [138]:
# df_dateall['datetime']=df_dateall.index

In [146]:
df_dateall

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER,datetime
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0,2022-01-01
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0,2022-01-02
2022-01-03,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2022-01-03
2022-01-04,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0,2022-01-04
2022-01-05,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2022-01-05
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27
2022-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28
2022-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30


In [145]:
# df_dateall.to_csv('./outdata/Krommenie_alldaylitter.csv')

In [62]:
df_dateall.max()

total_litter    466.0
SMOKING         290.0
FOOD            102.0
COFFEE           12.0
ALCOHOL          67.0
SOFTDRINKS       86.0
SANITARY         34.0
BRANDS          156.0
MATERIAL         19.0
OTHER           193.0
dtype: float64

## 天气时间数据处理

In [53]:
weather = pd.read_csv('./data/Hoorn/Hoorn_Weather.csv')

In [54]:
weather['date']=pd.to_datetime(weather['date'])

In [55]:
weather['weather'].value_counts()

Sunny      212
Cloudy     112
Showers     98
Foggy       30
Snow         4
Name: weather, dtype: int64

In [56]:
weather['weather'][weather['weather']=='Snow']='Foggy' # 最多4类的天气状况

In [57]:
weather['weather'].value_counts()

Sunny      212
Cloudy     112
Showers     98
Foggy       34
Name: weather, dtype: int64

In [58]:
wtcdt=pd.read_csv('./data/Hoorn/Hoorn.csv')

In [59]:
wtcdt['date']=pd.to_datetime(wtcdt['date'])

In [60]:
wtcdt = pd.DataFrame(wtcdt).set_index('date') # 将时间作为索引

In [61]:
wtcdt

,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
date,,,,,,,,
2021-01-01,43,30,39,28,100,87,8,1
2021-01-02,39,34,39,32,100,93,8,1
2021-01-03,39,37,37,32,100,75,16,5
2021-01-04,39,37,36,30,87,75,22,13
2021-01-05,39,36,36,32,93,81,16,10
...,...,...,...,...,...,...,...,...
2021-12-27,48,34,46,34,100,93,14,6
2021-12-28,50,46,48,43,100,76,23,9
2021-12-29,55,46,54,45,100,82,22,6


### 将天气状况加入补全后数据（舍弃）

In [ ]:
M_test=pd.concat([df_dateall,wtcdt],axis=1)# 横向合并

In [71]:
M_test

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0,54,52,50,50,88,88,12,12
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0,55,50,50,45,94,77,30,12
2022-01-03,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,52,46,46,45,93,76,25,10
2022-01-04,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0,48,37,46,32,93,70,16,5
2022-01-05,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,46,37,41,30,93,70,30,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46,41,43,36,93,71,22,9
2022-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,43,50,41,100,87,30,20
2022-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,45,48,41,93,76,29,15
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54,39,52,39,100,81,29,12


### 将天气状况加入未补全数据

In [62]:
# 匹配时间
df_dateconcat.index

Index([2021-01-06, 2021-01-07, 2021-01-08, 2021-01-09, 2021-01-10, 2021-01-11,
       2021-01-12, 2021-01-13, 2021-01-14, 2021-01-31,
       ...
       2021-12-11, 2021-12-12, 2021-12-13, 2021-12-15, 2021-12-16, 2021-12-17,
       2021-12-18, 2021-12-19, 2021-12-20, 2021-12-21],
      dtype='object', name='datetime', length=226)

In [63]:
isindex1=wtcdt.index.isin(df_dateconcat.index)

In [64]:
wtcdt[isindex1]

,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
date,,,,,,,,
2021-01-06,39,34,36,32,100,75,15,2
2021-01-07,41,34,37,32,100,81,16,5
2021-01-08,41,28,36,28,100,76,7,0
2021-01-09,43,25,37,25,100,76,8,1
2021-01-10,41,28,37,27,100,76,13,2
...,...,...,...,...,...,...,...,...
2021-12-17,48,45,46,45,100,87,9,0
2021-12-18,46,41,45,41,100,93,8,2
2021-12-19,46,43,45,39,100,81,12,3


In [65]:
isindex2=weather['date'].isin(df_dateconcat.index)

In [66]:
isindex2.value_counts()

False    230
True     226
Name: date, dtype: int64

In [67]:
wtdf = pd.DataFrame(weather[isindex2]).set_index('date')

In [68]:
M=pd.concat([df_dateconcat,wtdf,wtcdt[isindex1]],axis=1)# 横向合并

In [69]:
M

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER,weather,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
2021-01-06,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Sunny,39,34,36,32,100,75,15,2
2021-01-07,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,Showers,41,34,37,32,100,81,16,5
2021-01-08,114.0,2.0,13.0,6.0,3.0,34.0,9.0,0.0,25.0,32.0,Showers,41,28,36,28,100,76,7,0
2021-01-09,58.0,3.0,11.0,1.0,0.0,12.0,3.0,0.0,11.0,21.0,Foggy,43,25,37,25,100,76,8,1
2021-01-10,49.0,2.0,2.0,0.0,8.0,20.0,0.0,0.0,18.0,9.0,Cloudy,41,28,37,27,100,76,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-17,140.0,31.0,14.0,5.0,1.0,36.0,8.0,0.0,22.0,45.0,Showers,48,45,46,45,100,87,9,0
2021-12-18,18.0,0.0,6.0,0.0,3.0,6.0,1.0,0.0,6.0,2.0,Foggy,46,41,45,41,100,93,8,2
2021-12-19,92.0,19.0,10.0,1.0,5.0,28.0,7.0,0.0,28.0,22.0,Showers,46,43,45,39,100,81,12,3
2021-12-20,232.0,5.0,22.0,7.0,7.0,60.0,17.0,0.0,50.0,114.0,Cloudy,45,32,39,30,93,65,7,0


In [67]:
# M_train=M.drop(M.loc[:,'SMOKING':'OTHER'],axis=1)# 舍弃分类

### 对天气描述编码

In [71]:
# M['weather']

2021-01-06      Sunny
2021-01-07    Showers
2021-01-08    Showers
2021-01-09      Foggy
2021-01-10     Cloudy
               ...   
2021-12-17    Showers
2021-12-18      Foggy
2021-12-19    Showers
2021-12-20     Cloudy
2021-12-21      Sunny
Name: weather, Length: 226, dtype: object

In [69]:
# weather = pd.DataFrame(weather).set_index('date') # 将时间作为索引

In [72]:
wtdf

,weather
date,
2021-01-06,Sunny
2021-01-07,Showers
2021-01-08,Showers
2021-01-09,Foggy
2021-01-10,Cloudy
...,...
2021-12-17,Showers
2021-12-18,Foggy
2021-12-19,Showers


In [73]:
values_wt=wtdf['weather']

In [74]:
# 整数编码
label_encoder_wt = LabelEncoder()
integer_encoded_wt = label_encoder_wt.fit_transform(values_wt)
integer_encoded_wt

array([3, 2, 2, 1, 0, 0, 2, 3, 3, 3, 2, 2, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 0, 3, 0, 2, 3, 0, 2, 0, 0, 0, 0, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0, 0,
       3, 2, 3, 0, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3,
       3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 3, 2, 0,
       3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3,
       3, 3, 0, 2, 0, 3, 3, 1, 0, 2, 3, 2, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 2, 0, 2, 2, 2, 3, 3, 0, 1, 3,
       1, 3, 2, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 3, 3, 2, 1, 1, 0, 0, 0,
       0, 1, 2, 0, 1, 3, 0, 2, 3, 0, 2, 2, 2, 1, 1, 2, 3, 2, 3, 2, 0, 0,
       1, 2, 1, 2, 0, 3])

In [75]:
# 二进制编码
wt_encoded = OneHotEncoder(sparse=False)
integer_encoded_wt = integer_encoded_wt.reshape(len(integer_encoded_wt), 1)
wt_encoded = wt_encoded.fit_transform(integer_encoded_wt)
wt_encoded.shape

(226, 4)

In [76]:
# 第一个例子的反转
inverted_wt = label_encoder_wt.inverse_transform([argmax(wt_encoded[0, :])])
inverted_wt

array(['Sunny'], dtype=object)

### 天气特征归一化(舍弃)

In [78]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [79]:
M_train.loc[:,'Actual Mean Temperature':'Actual Sea Level Pressure'].values.reshape(-1, 1)

array([[36.59],
       [34.  ],
       [39.  ],
       ...,
       [36.  ],
       [27.  ],
       [30.58]])

In [80]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(M_train.loc[:,'Actual Mean Temperature':'Actual Sea Level Pressure'].values.reshape(-1, 1))

In [81]:
mm_weather=mm_data.reshape(228,10)
mm_weather

array([[0.43559524, 0.4047619 , 0.46428571, ..., 0.42857143, 0.38095238,
        0.35809524],
       [0.44690476, 0.4047619 , 0.48809524, ..., 0.44047619, 0.38095238,
        0.35666667],
       [0.42809524, 0.35714286, 0.48809524, ..., 0.42857143, 0.35714286,
        0.35869048],
       ...,
       [0.37857143, 0.32142857, 0.51190476, ..., 0.38095238, 0.32142857,
        0.36154762],
       [0.47130952, 0.42857143, 0.51190476, ..., 0.44047619, 0.33333333,
        0.36333333],
       [0.49380952, 0.42857143, 0.57142857, ..., 0.42857143, 0.32142857,
        0.36404762]])

In [232]:
# 合并归一化后的数据
mm_weather_df=pd.DataFrame(mm_weather)
mm_weather_df=mm_weather_df.set_index(wtdf.index)
wt_encoded_df=pd.DataFrame(wt_encoded)
wt_encoded_df=wt_encoded_df.set_index(wtdf.index)
ML=pd.concat([wt_encoded_df,mm_weather_df,M_train['total_litter']],axis=1)

In [233]:
ML

,0,1,2,3,0,1,2,3,4,5,6,7,8,9,total_litter
2021-01-06,0.0,0.0,0.0,1.0,0.435595,0.404762,0.464286,0.369048,0.476190,0.178571,0.388214,0.428571,0.380952,0.358095,3.0
2021-01-07,0.0,0.0,1.0,0.0,0.446905,0.404762,0.488095,0.369048,0.476190,0.190476,0.420595,0.440476,0.380952,0.356667,2.0
2021-01-08,0.0,0.0,1.0,0.0,0.428095,0.357143,0.488095,0.369048,0.476190,0.083333,0.405833,0.428571,0.357143,0.358690,114.0
2021-01-09,0.0,1.0,0.0,0.0,0.386071,0.297619,0.511905,0.369048,0.476190,0.095238,0.352024,0.440476,0.297619,0.360476,58.0
2021-01-10,1.0,0.0,0.0,0.0,0.431429,0.333333,0.488095,0.369048,0.476190,0.142857,0.393690,0.440476,0.321429,0.360476,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,0.0,0.0,1.0,0.0,0.536548,0.511905,0.547619,0.392857,0.500000,0.142857,0.518690,0.535714,0.488095,0.364762,92.0
2021-12-20,1.0,0.0,0.0,0.0,0.495595,0.380952,0.535714,0.392857,0.500000,0.083333,0.418690,0.464286,0.357143,0.361905,232.0
2021-12-21,0.0,0.0,0.0,1.0,0.378571,0.321429,0.511905,0.392857,0.500000,0.059524,0.339762,0.380952,0.321429,0.361548,14.0
2022-01-20,1.0,0.0,0.0,0.0,0.471310,0.428571,0.511905,0.369048,0.476190,0.357143,0.392024,0.440476,0.333333,0.363333,5.0


In [84]:
ML_array=np.array(ML)

In [85]:
np.save('Hoorn_array', ML)

## poi分类统计（舍弃）

In [55]:
points=gpd.read_file('./data/Hoorn/shape_large/points.shp')

In [56]:
points

,osm_id,timestamp,name,type,geometry
0,34050780,None,Museum van de Twintigste Eeuw,museum,POINT (5.06242 52.63508)
1,47051205,None,Avenhorn,motorway_junctio,POINT (5.00851 52.63502)
2,47056631,None,None,crossing,POINT (5.05786 52.63695)
3,47060139,None,None,crossing,POINT (5.05724 52.63817)
4,47069901,None,None,crossing,POINT (5.05239 52.64179)
...,...,...,...,...,...
2397,804881165,None,None,traffic_signals,POINT (5.04726 52.64767)
2398,804881166,None,None,traffic_signals,POINT (5.04692 52.64760)
2399,804881167,None,None,traffic_signals,POINT (5.04659 52.64766)
2400,804881168,None,None,traffic_signals,POINT (5.04734 52.64798)


In [117]:
public=points['type'].isin(['post_box', 'pub',  'school', 'social_facility','charging_station','childcare','arts_centre',

'community_centre','kindergarten','library',  'elevator','fire_station','post_depot', 

 'social_centre', 'marketplace','police', 'theatre','yard','music_school','townhall'])

In [118]:
poi_public=points['type'][public].value_counts()

In [119]:
poi_public=poi_public.values.sum()

In [120]:
poi_public

53

In [121]:
points['type'][public]='public'

In [122]:
health=points['type'].isin(['dentist','doctors','pharmacy', 'veterinary','hospital', 'clinic'])

In [123]:
poi_health=points['type'][health].value_counts()

In [124]:
poi_health=poi_health.values.sum()

In [125]:
points['type'][health]='health'

In [126]:
leisure=points['type'].isin(['cafe', 'cinema','casino'])

In [127]:
poi_leisure=points['type'][leisure].value_counts()

In [128]:
poi_leisure=poi_leisure.values.sum()

In [129]:
poi_leisure

8

In [130]:
points['type'][leisure]='leisure'

In [131]:
catering=points['type'].isin([ 'restaurant','fast_food'])

In [132]:
poi_catering=points['type'][catering].value_counts()

In [133]:
poi_catering=poi_catering.values.sum()

In [134]:
poi_catering

28

In [135]:
points['type'][catering]='catering'

In [136]:
accommodation=points['type'].isin(['hotel','shelter','caravan_site'])

In [137]:
poi_accommodation=points['type'][accommodation].value_counts()

In [138]:
poi_accommodation=poi_accommodation.values.sum()

In [139]:
poi_accommodation

0

In [140]:
points['type'][accommodation]='accommodation'

In [141]:
shopping=points['type'].isin(['car_wash','vending_machine', 'car_sharing','ice_cream','parking', 'shower','bicycle_repair_s', 'car_rental','taxi'])

In [142]:
poi_shopping=points['type'][shopping].value_counts()

In [143]:
poi_shopping=poi_shopping.values.sum()

In [144]:
poi_shopping

6

In [145]:
points['type'][shopping]='shopping'

In [146]:
money=points['type'].isin(['atm','bank'])

In [147]:
poi_money=points['type'][money].value_counts()

In [148]:
poi_money=poi_money.values.sum()

In [149]:
poi_money

4

In [150]:
points['type'][money]='money'

In [151]:
tourism=points['type'].isin(['artwork', 'memorial', 'gallery','museum','attraction', 'cannon',  'archaeological_s',  'viewpoint', 'picnic_site'])

In [152]:
poi_tourism=points['type'][tourism].value_counts()

In [153]:
poi_tourism=poi_tourism.values.sum()

In [154]:
poi_tourism

7

In [155]:
points['type'][tourism]='tourism'

In [156]:
miscpoi=points['type'].isin(['bench','waste_basket','tower','toilets','surveillance','works','drinking_water', 'windmill','fountain', 'water_well'])

In [157]:
poi_miscpoi=points['type'][miscpoi].value_counts()

In [158]:
poi_miscpoi=poi_miscpoi.values.sum()

In [159]:
poi_miscpoi

34

In [160]:
points['type'][miscpoi]='miscpoi'

In [161]:
other=points['type'].isin(['construction','give_way','clock','street_cabinet', 'studio', 'place_of_worship',  'crane',  'communications_t','flagpole', 'chimney', 'lighthouse','tram_stop', 'yes'])

In [162]:
poi_other=points['type'][other].value_counts()

In [163]:
poi_other=poi_other.values.sum()

In [164]:
poi_other

4

In [165]:
points['type'][other]='other'

In [166]:
traffic=points['type'].isin(['mast','bicycle_parking','crossing', 'recycling', 'traffic_signals', 'signal','switch', 'level_crossing','motorway_junctio','ferry_terminal','turning_circle', 'buffer_stop', 'street_lamp','fuel', 'parking_entrance', 'stop', 'station','milestone','mini_roundabout',  'railway_crossing','junction'])

In [167]:
poi_traffic=points['type'][traffic].value_counts()

In [168]:
poi_traffic=poi_traffic.values.sum()

In [169]:
poi_traffic

207

In [170]:
points['type'][traffic]='traffic'

In [182]:
poi=pd.DataFrame(np.array([poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other]).reshape(1,10),
                 columns=['poi_public','poi_health','poi_leisure','poi_catering','poi_shopping','poi_money','poi_tourism','poi_miscpoi','poi_traffic','poi_other'],
                 index=['counter']
                )# 去除数量为0的poi

In [183]:
poi

,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
counter,53,17,8,28,6,4,7,34,207,4


In [173]:
# poi_other=poi_other+poi_money+poi_shopping+poi_accommodation+poi_health+poi_leisure

In [144]:
# poi['poi_other']=poi_other

In [145]:
# poi=poi.drop(['poi_health','poi_leisure','poi_accommodation','poi_shopping','poi_money'],axis=1)

In [146]:
# poi

In [184]:
poi_array=np.array(poi)

In [185]:
poi_array=np.tile(poi,(249,1))

In [186]:
poi_array

array([[ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       ...,
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4]], dtype=int64)

In [187]:
poi_df=pd.DataFrame(poi_array,columns=poi.columns,index=M_train.index)

In [188]:
poi_df

,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2022-01-01,53,17,8,28,6,4,7,34,207,4
2022-01-02,53,17,8,28,6,4,7,34,207,4
2022-01-07,53,17,8,28,6,4,7,34,207,4
2022-01-10,53,17,8,28,6,4,7,34,207,4
2022-01-11,53,17,8,28,6,4,7,34,207,4
...,...,...,...,...,...,...,...,...,...,...
2022-12-25,53,17,8,28,6,4,7,34,207,4
2022-12-27,53,17,8,28,6,4,7,34,207,4
2022-12-29,53,17,8,28,6,4,7,34,207,4
2022-12-30,53,17,8,28,6,4,7,34,207,4


In [189]:
M_train=pd.concat([M_train,poi_df],axis=1)

In [190]:
M_train

,total_litter,weather,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2022-01-01,57.0,Showers,54,52,50,50,88,88,12,12,53,17,8,28,6,4,7,34,207,4
2022-01-02,48.0,Showers,55,50,50,45,94,77,30,12,53,17,8,28,6,4,7,34,207,4
2022-01-07,2.0,Showers,45,36,39,32,93,70,24,5,53,17,8,28,6,4,7,34,207,4
2022-01-10,41.0,Cloudy,45,32,41,30,100,81,9,2,53,17,8,28,6,4,7,34,207,4
2022-01-11,2.0,Foggy,41,32,39,32,100,76,14,6,53,17,8,28,6,4,7,34,207,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-25,229.0,Showers,48,45,48,45,100,93,12,5,53,17,8,28,6,4,7,34,207,4
2022-12-27,152.0,Cloudy,46,41,43,36,93,71,22,9,53,17,8,28,6,4,7,34,207,4
2022-12-29,160.0,Showers,52,45,48,41,93,76,29,15,53,17,8,28,6,4,7,34,207,4
2022-12-30,228.0,Showers,54,39,52,39,100,81,29,12,53,17,8,28,6,4,7,34,207,4


In [191]:
M_train.columns

Index(['total_litter', 'weather', 'Max Temperature', 'Min Temperature',
       'Max Dew Point', 'Min Dew Point', 'Max Humidity', 'Min Humidity',
       'Max Wind Speed', 'Min Wind Speed', 'poi_public', 'poi_health',
       'poi_leisure', 'poi_catering', 'poi_shopping', 'poi_money',
       'poi_tourism', 'poi_miscpoi', 'poi_traffic', 'poi_other'],
      dtype='object')

## 将数值型数据归一化

In [77]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [76]:
# M_test=M_test.drop(M_test.loc[:,'SMOKING':'OTHER'],axis=1)# 舍弃分类

In [78]:
M = M.drop(columns=['weather'])

In [84]:
M.loc[:,'SMOKING':'Min Wind Speed'].values.reshape(-1,1)

array([[ 0.],
       [ 2.],
       [ 0.],
       ...,
       [61.],
       [ 5.],
       [ 1.]])

In [85]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(M.loc[:,'SMOKING':'Min Wind Speed'].values.reshape(-1,1))

In [86]:
mm_wp=mm_data.reshape(226,17)

In [87]:
Counter(mm_wp.flatten())

Counter({0.0: 487,
         0.010526315789473684: 172,
         0.005263157894736842: 231,
         0.20526315789473684: 49,
         0.17894736842105263: 37,
         0.18947368421052632: 50,
         0.16842105263157894: 44,
         0.5263157894736842: 123,
         0.39473684210526316: 10,
         0.07894736842105263: 65,
         0.21578947368421053: 48,
         0.19473684210526315: 38,
         0.4263157894736842: 9,
         0.08421052631578947: 51,
         0.02631578947368421: 90,
         0.06842105263157895: 53,
         0.031578947368421054: 98,
         0.015789473684210527: 142,
         0.04736842105263158: 64,
         0.13157894736842105: 31,
         0.14736842105263157: 35,
         0.39999999999999997: 15,
         0.03684210526315789: 89,
         0.05789473684210526: 34,
         0.06315789473684211: 51,
         0.11052631578947368: 31,
         0.2263157894736842: 54,
         0.042105263157894736: 78,
         0.10526315789473684: 34,
         0.0947368421052

In [88]:
# 合并归一化后的数据
mm_wp_df=pd.DataFrame(mm_wp)
mm_wp_df=mm_wp_df.set_index(M.index)
wt_encoded_df=pd.DataFrame(wt_encoded)
wt_encoded_df=wt_encoded_df.set_index(M.index)
ML=pd.concat([wt_encoded_df,mm_wp_df,M['total_litter']],axis=1)

In [89]:
ML

,0,1,2,3,0,1,2,3,4,5,...,8,9,10,11,12,13,14,15,16,total_litter
2021-01-06,0.0,0.0,0.0,1.0,0.000000,0.010526,0.000000,0.000000,0.005263,0.000000,...,0.000000,0.205263,0.178947,0.189474,0.168421,0.526316,0.394737,0.078947,0.010526,3.0
2021-01-07,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.005263,0.000000,...,0.000000,0.215789,0.178947,0.194737,0.168421,0.526316,0.426316,0.084211,0.026316,2.0
2021-01-08,0.0,0.0,1.0,0.0,0.010526,0.068421,0.031579,0.015789,0.178947,0.047368,...,0.168421,0.215789,0.147368,0.189474,0.147368,0.526316,0.400000,0.036842,0.000000,114.0
2021-01-09,0.0,1.0,0.0,0.0,0.015789,0.057895,0.005263,0.000000,0.063158,0.015789,...,0.110526,0.226316,0.131579,0.194737,0.131579,0.526316,0.400000,0.042105,0.005263,58.0
2021-01-10,1.0,0.0,0.0,0.0,0.010526,0.010526,0.000000,0.042105,0.105263,0.000000,...,0.047368,0.215789,0.147368,0.194737,0.142105,0.526316,0.400000,0.068421,0.010526,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-17,0.0,0.0,1.0,0.0,0.163158,0.073684,0.026316,0.005263,0.189474,0.042105,...,0.236842,0.252632,0.236842,0.242105,0.236842,0.526316,0.457895,0.047368,0.000000,140.0
2021-12-18,0.0,1.0,0.0,0.0,0.000000,0.031579,0.000000,0.015789,0.031579,0.005263,...,0.010526,0.242105,0.215789,0.236842,0.215789,0.526316,0.489474,0.042105,0.010526,18.0
2021-12-19,0.0,0.0,1.0,0.0,0.100000,0.052632,0.005263,0.026316,0.147368,0.036842,...,0.115789,0.242105,0.226316,0.236842,0.205263,0.526316,0.426316,0.063158,0.015789,92.0
2021-12-20,1.0,0.0,0.0,0.0,0.026316,0.115789,0.036842,0.036842,0.315789,0.089474,...,0.600000,0.236842,0.168421,0.205263,0.157895,0.489474,0.342105,0.036842,0.000000,232.0


In [84]:
# ML_array=np.array(ML)

In [87]:
# ML.to_csv('./outdata/Krommenie_train.csv')

## poi提取(舍弃）

In [164]:
poi1=pd.read_csv('./outdata/poi1.csv')

In [165]:
poi1['geometry']

0      POINT (5.0817843 52.6456291)
1      POINT (5.0961325 52.6457239)
2      POINT (5.0812344 52.6458317)
3      POINT (5.0968648 52.6460559)
4      POINT (5.0853898 52.6531698)
                   ...             
450    POINT (5.0931458 52.6533975)
451    POINT (5.0932666 52.6531166)
452    POINT (5.0932798 52.6532018)
453    POINT (5.0934684 52.6531761)
454    POINT (5.0936391 52.6532932)
Name: geometry, Length: 455, dtype: object

In [166]:
points['geometry'].astype(str)

0       POINT (5.0624231 52.6350804)
1       POINT (5.0085054 52.6350184)
2         POINT (5.0578571 52.63695)
3       POINT (5.0572411 52.6381662)
4       POINT (5.0523931 52.6417891)
                    ...             
2397     POINT (5.0472634 52.647675)
2398    POINT (5.0469239 52.6475957)
2399    POINT (5.0465883 52.6476558)
2400    POINT (5.0473351 52.6479847)
2401    POINT (5.0471709 52.6476163)
Name: geometry, Length: 2402, dtype: object

In [167]:
poi1_flag=points['geometry'].astype(str).isin(poi1['geometry'])

In [168]:
poi1_flag.value_counts()

False    1947
True      455
Name: geometry, dtype: int64

In [169]:
poi1=points[poi1_flag].reset_index(drop=True)# 第一类poi

In [170]:
poi1.to_file('./poi1.shp')# 输出为shapefile文件

In [171]:
poi1[['osm_id','type','geometry']].set_index(['osm_id'])

,type,geometry
osm_id,,
47077200,traffic,POINT (5.08178 52.64563)
47080156,traffic,POINT (5.09613 52.64572)
47081177,traffic,POINT (5.08123 52.64583)
47081536,traffic,POINT (5.09686 52.64606)
47097804,traffic,POINT (5.08539 52.65317)
...,...,...
49020516,miscpoi,POINT (5.09315 52.65340)
49020522,miscpoi,POINT (5.09327 52.65312)
49020523,miscpoi,POINT (5.09328 52.65320)


## 时间分段标签

In [90]:
df_date=pd.to_datetime(df_dateconcat.index)

In [91]:
df_date

DatetimeIndex(['2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09',
               '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13',
               '2021-01-14', '2021-01-31',
               ...
               '2021-12-11', '2021-12-12', '2021-12-13', '2021-12-15',
               '2021-12-16', '2021-12-17', '2021-12-18', '2021-12-19',
               '2021-12-20', '2021-12-21'],
              dtype='datetime64[ns]', name='datetime', length=226, freq=None)

In [92]:
is_weekly=df_date.weekday==(5 or 6) # 一周结束

In [93]:
df_date.weekday==(5 or 6)

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
        True, False,

In [94]:
holiday=pd.to_datetime(['2021-01-01','2021-04-02','2021-04-04','2021-04-27','2021-05-04','2021-05-05','2021-05-13','2021-05-23','2021-05-24',
         '2021-12-05','2021-12-06','2021-12-24','2021-12-25','2021-12-26','2021-12-31',
        '2022-01-01','2022-04-02','2022-04-04','2022-04-27','2022-05-04','2022-05-05','2022-05-26','2022-06-05','2022-06-06',
         '2022-12-05','2022-12-06','2022-12-24','2022-12-25','2022-12-26','2022-12-31'])

In [95]:
is_holiday=df_date.isin(holiday)

In [96]:
is_holiday=np.array(is_holiday)+np.array(df_date.weekday==(5 or 6))

In [100]:
Date2=pd.DataFrame(np.zeros([226,3]),columns=['daily','weekly','holiday'])

In [101]:
Date2=Date2.set_index(df_date.date)

In [102]:
ML1=pd.concat([Date2,ML],axis=1)

In [103]:
ML1

,daily,weekly,holiday,0,1,2,3,0,1,2,...,8,9,10,11,12,13,14,15,16,total_litter
2021-01-06,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.010526,0.000000,...,0.000000,0.205263,0.178947,0.189474,0.168421,0.526316,0.394737,0.078947,0.010526,3.0
2021-01-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.215789,0.178947,0.194737,0.168421,0.526316,0.426316,0.084211,0.026316,2.0
2021-01-08,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.010526,0.068421,0.031579,...,0.168421,0.215789,0.147368,0.189474,0.147368,0.526316,0.400000,0.036842,0.000000,114.0
2021-01-09,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.015789,0.057895,0.005263,...,0.110526,0.226316,0.131579,0.194737,0.131579,0.526316,0.400000,0.042105,0.005263,58.0
2021-01-10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.010526,0.010526,0.000000,...,0.047368,0.215789,0.147368,0.194737,0.142105,0.526316,0.400000,0.068421,0.010526,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-17,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.163158,0.073684,0.026316,...,0.236842,0.252632,0.236842,0.242105,0.236842,0.526316,0.457895,0.047368,0.000000,140.0
2021-12-18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.031579,0.000000,...,0.010526,0.242105,0.215789,0.236842,0.215789,0.526316,0.489474,0.042105,0.010526,18.0
2021-12-19,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.100000,0.052632,0.005263,...,0.115789,0.242105,0.226316,0.236842,0.205263,0.526316,0.426316,0.063158,0.015789,92.0
2021-12-20,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.026316,0.115789,0.036842,...,0.600000,0.236842,0.168421,0.205263,0.157895,0.489474,0.342105,0.036842,0.000000,232.0


In [104]:
ML1['daily']=1.0

In [105]:
ML1[is_weekly].index

Index([2021-01-09, 2021-02-20, 2021-02-27, 2021-04-03, 2021-04-17, 2021-04-24,
       2021-05-01, 2021-05-08, 2021-05-15, 2021-06-05, 2021-06-12, 2021-06-19,
       2021-06-26, 2021-07-03, 2021-07-17, 2021-07-24, 2021-08-21, 2021-08-28,
       2021-09-04, 2021-09-18, 2021-09-25, 2021-10-02, 2021-10-09, 2021-10-16,
       2021-10-23, 2021-11-06, 2021-11-13, 2021-11-20, 2021-11-27, 2021-12-04,
       2021-12-11, 2021-12-18],
      dtype='object')

In [106]:
ML1.loc[is_weekly,'weekly']=1.0

In [107]:
ML1['weekly'].value_counts()

0.0    194
1.0     32
Name: weekly, dtype: int64

In [108]:
ML1.loc[is_holiday,'holiday']=1.0

In [109]:
ML1['holiday'].value_counts()

0.0    190
1.0     36
Name: holiday, dtype: int64

In [110]:
ML1

,daily,weekly,holiday,0,1,2,3,0,1,2,...,8,9,10,11,12,13,14,15,16,total_litter
2021-01-06,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.010526,0.000000,...,0.000000,0.205263,0.178947,0.189474,0.168421,0.526316,0.394737,0.078947,0.010526,3.0
2021-01-07,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.215789,0.178947,0.194737,0.168421,0.526316,0.426316,0.084211,0.026316,2.0
2021-01-08,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.010526,0.068421,0.031579,...,0.168421,0.215789,0.147368,0.189474,0.147368,0.526316,0.400000,0.036842,0.000000,114.0
2021-01-09,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.015789,0.057895,0.005263,...,0.110526,0.226316,0.131579,0.194737,0.131579,0.526316,0.400000,0.042105,0.005263,58.0
2021-01-10,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.010526,0.010526,0.000000,...,0.047368,0.215789,0.147368,0.194737,0.142105,0.526316,0.400000,0.068421,0.010526,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-17,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.163158,0.073684,0.026316,...,0.236842,0.252632,0.236842,0.242105,0.236842,0.526316,0.457895,0.047368,0.000000,140.0
2021-12-18,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.000000,0.031579,0.000000,...,0.010526,0.242105,0.215789,0.236842,0.215789,0.526316,0.489474,0.042105,0.010526,18.0
2021-12-19,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.100000,0.052632,0.005263,...,0.115789,0.242105,0.226316,0.236842,0.205263,0.526316,0.426316,0.063158,0.015789,92.0
2021-12-20,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.026316,0.115789,0.036842,...,0.600000,0.236842,0.168421,0.205263,0.157895,0.489474,0.342105,0.036842,0.000000,232.0


In [112]:
ML1.to_csv('./outdata/Hoorn_ML.csv')